<a href="https://colab.research.google.com/github/cspinheiro/deep-learning/blob/main/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conhecimento Natural na GoBots: Extraindo Informações de Títulos de Anúncios


In [ ]:
!pip install spacy --upgrade
#!pip install spacy==2.2.3

# Etapa 1: Instalação do spaCy

In [2]:
import spacy
spacy.__version__

'2.2.4'

In [3]:
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.3MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186283 sha256=57834d45e08baa5e3af2d34195654f88ef4d4f373de5c007e713468fa88577bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-zcxkff5x/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
from google.colab import files 
uploaded = files.upload()

# Etapa 2: Marcação POS

- POS (part-of-speech) atribui para as palavras partes da fala, como substantivos, adjetivos, verbos
- Importante para a detecção de entidades no texto, pois primeiro é necessário saber o que o texto contém
- Lista de tokens: https://spacy.io/api/annotation#pos-tagging
- Português: https://www.sketchengine.eu/portuguese-freeling-part-of-speech-tagset/

In [5]:
pln = spacy.load('pt')
pln

In [7]:
documento = pln('Meu nome é Carolinne e falo em nome do Banco Santander')

In [8]:
for token in documento:
  print(token.text, token.pos_)

Meu DET
nome NOUN
é VERB
Carolinne PROPN
e CCONJ
falo ADV
em ADP
nome NOUN
do PROPN
Banco PROPN
Santander PROPN


## Legenda 

- lemma: raiz da palavra
- pos: parte da fala
- tag: informações morfológicas, como se o verbo está no passado
- dep: dependência sintática
- shape: formato (maiúsculo, minúsculo, dígitos)
- alpha: se é alfabético
- stop: se é stopword

In [ ]:
for token in documento:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, 
        token.shape_, token.is_alpha, token.is_stop)

In [ ]:
for token in documento:
  if token.pos_ == 'PROPN':
    print(token.text)

# Etapa 3: Lematização e stemização

- Lematização: "Lema" de uma palavra de acordo com seu significado no dicionário - palavra base (análise vocabular e morfológica)
- Stemização: Extrair o radical das palavras

In [9]:
for token in documento:
  print(token.text, token.lemma_)

Meu Meu
nome nome
é ser
Carolinne Carolinne
e e
falo falir
em em
nome nome
do do
Banco Banco
Santander Santander


In [10]:
doc = pln('encontrei encontraram encontrarão encontrariam cursando curso cursei')
[token.lemma_ for token in doc]

['encontrar',
 'encontrar',
 'encontrar',
 'encontrar',
 'cursar',
 'cursar',
 'cursar']

## Comparação stemização (NLTK) x lematização (spaCy)

In [ ]:
#!pip install nltk --upgrade

In [ ]:
import nltk
nltk.download('rslp')

In [ ]:
stemmer = nltk.stem.RSLPStemmer()
stemmer.stem('aprender')

'aprend'

In [ ]:
for token in documento:
  print(token.text, token.lemma_, stemmer.stem(token.text))

# Etapa 4: Reconhecimento de entidades nomeadas

- NER (Named-Entity Recognition)
- Encontrar e classificar entidades no texto, dependendo da base de dados que foi utilizada para o treinamento (pessoa, localização, empresa, numéricos)
- Usado em chatbots para saber o assunto falado
- Siglas: https://spacy.io/api/annotation#named-entities

In [11]:
texto = 'Meu nome é Carolinne e falo em nome da empresa Santander'

In [13]:
documento = pln(texto)

In [14]:
for entidade in documento.ents:
  print(entidade.text, entidade.label_)

Carolinne LOC
Santander MISC


In [15]:
from spacy import displacy
displacy.render(documento, style = 'ent', jupyter = True)

In [21]:
texto = 'Olá, bom dia, sou o Victor, da Flex, São Paulo, e falo em nome do Santander'

In [22]:
documento = pln(texto)
for entidade in documento.ents:
  print(entidade.text, entidade.label_)

Victor PER
Flex ORG
São Paulo LOC
Santander MISC


In [23]:
displacy.render(documento, style = 'ent', jupyter = True)

In [24]:
for entidade in documento.ents:
  if entidade.label_ == 'PER':
    print(entidade.text)

Victor


# Etapa 5: Stopwords

- Palavras que aparecem com muita frequência e que não apresentam muito significado (e, a, de, da, etc)

In [ ]:
from spacy.lang.pt.stop_words import STOP_WORDS

In [ ]:
print(STOP_WORDS)

In [ ]:
len(STOP_WORDS)

In [ ]:
pln.vocab['ir'].is_stop

In [ ]:
pln.vocab['caminhar'].is_stop

In [ ]:
documento = pln('Celular ASUS black 8GB de memória à pronta entrega em São Paulo')

In [ ]:
for token in documento:
  if not pln.vocab[token.text].is_stop:
    print(token.text)

# Etapa 6: Semelhanças entre palavras e frases

- Verificar se duas palavras são semelhantes ou logicamente relacionadas
- Usa o algoritmo GloVe (Global Vectors for Word Representation)
- Artigo original: https://nlp.stanford.edu/pubs/glove.pdf

## Exemplo 1

In [25]:
p1 = pln("olá")
p2 = pln("oi")
p3 = pln("ou")

In [ ]:
p1.similarity(p2)

In [ ]:
p2.similarity(p1)

In [ ]:
p1.similarity(p3)

In [ ]:
p2.similarity(p3)

In [ ]:
texto1 = pln('quanto custa o celular?')
texto2 = pln('qual é o preço do celular')
texto3 = pln('vem com fone?')

In [ ]:
texto1.similarity(texto2)

In [ ]:
texto1.similarity(texto3)

## Exemplo 2

In [ ]:
texto = pln('Camiseta Gola V Masculina Básica Lisa Algodão')

In [ ]:
for texto1 in texto:
  #print('----', texto1)
  for texto2 in texto:
    #print(texto2)
    similaridade = int(texto1.similarity(texto2) * 100)
    print("{} é {} similar a {}".format(texto1, similaridade, texto2))

# Etapa 7: Tokenização

In [ ]:
documento = pln('Camiseta Gola V Masculina Básica Lisa Algodão Camisa Blusa')

In [ ]:
for token in documento:
  print(token)

In [ ]:
documento1 = 'Camiseta Gola V Masculina Básica Lisa Algodão Camisa Blusa'
documento1.split(' ')